# This notebook goes over the functions that live in LabelWriter

Majority of the functions here have been created as quality of life to help create storage files, labels and simplify recursive runs

Ensure that the version of mapper matches the latest version available as documented in the read me file

In [ ]:
#mapper version verification
!pip show views_mapper2

In [ ]:
#import the portion of views_mapper2 that you are interested in
from views_mapper2.label_writer import *

In [ ]:
#create a reference for this run, we will store everything on desktop for now
import os
home = os.path.expanduser("~")
desktop = home+'/Desktop'

## folder creation

In [ ]:
#function 1, create a single folder

#this function creates a folder of interest
#specify folderpath as a string and use make_a_folder(string) function
#note that a warning message will pop up if you try to create a folder that already exists

working_folder = desktop + '/mapper2_showcase_delete_when_done'
make_a_folder(working_folder)

In [ ]:
#function 2, create a set of folder for monthly updates
#function make_folders_complete_set creates a set of pre-determined folders for MonthlyUpdates
#only need to specify the head folderpath/name and the subfolders within it will be created automatically
#this function only has a use for MonthlyUpdateTeam
monthly_update_example = working_folder + '/monthly_update_example'
make_folders_complete_set(monthly_update_example)

## Date look up, date conversions

Several functions are available for writing dates in various formats from month_id (time identification in viewser)
Or to convert dates from ISO format to month_id
This is useful for quick lookup of information or for recursive code

In [ ]:
#function 1, gives a date in year/month format from a month_id, useful for notation in graphs/maps
vid2date(509)

In [ ]:
#function 2, gives a date in year_month format from month_id, useful for savefile notation
vid2date_version2(509)

In [ ]:
#function 3, easy lookup of month_id if you know date
#this is a quality of life function that can sometimes be useful during data analysis
#write date in ISO format as 'YEAR-MONTH-DAY'
#note that day is not necessary but was needed to stick to iso format, use day=1

date2id('2022-05-01')

## Country look up, country conversions

Similar to dates, functionalities to look up country names or visa versa have been created
Please note that since country_id is associated with month_id in viewser, one needs to input both

In [ ]:
#function 1, country id to name
#useful for automatic notation creation in tables/maps/graphs or for quick lookup of countries
cid2name(30)

In [ ]:
#function 2, country name to iso conversion
#note that this function has a built in grey search for potential options that you might mean
#success of suggestions is variable as seen below in Ivory Coast, UK, and America examples
#if no matches are found and no grey options, check_spelling message appears

display(name2iso('Cuba'))
display(name2iso("Cote d'Ivoire"))
display(name2iso('Great Britain'))
display(name2iso('England'))
display(name2iso('America'))
display(name2iso('Narnia'))

In [ ]:
#function 3, convert name and month id into country_id
#in case you get the name incorrectly same functionality as above is applied
#date needs to be written in standard iso format year-month-day

display(name_date2cid('Cuba', '2022-05-01'))
display(name_date2cid("Cote d'Ivoire", '2022-05-01'))
display(name_date2cid('Narnia', '2022-05-01'))

## Give me top X countries

These functions can either be used inside other loops to for example create loops that make maps of the highest top 10 fatality countries or simply identifies them for your reference

In [ ]:
#grab some data for experimentation
from viewser import Queryset, Column

queryset = (Queryset("Mapper_tester_cm", "country_month")
            # CONFLICT TYPES NOT LOGGED and NOT time lagged
            .with_column(Column("ged_best_sb", from_table="ged2_cm", from_column="ged_sb_best_sum_nokgi")
                         .transform.missing.fill()
                         )
            # .with_column(Column('name', from_table='country', from_column= 'name')
            # the above does not seem to want to work

            .with_theme("fatalities")

            .describe("""Fatalities conflict history, cm level
            Mapper testing

            """)

            )

cm_master = queryset.publish().fetch()

In [ ]:
#function 1, function that gives top 10 country_id in a given month
#this can be useful for embedding/creating loops

give_me_top10_country_id(df = cm_master, month_value = 510, var_name = 'ged_best_sb')

In [ ]:
#function 2, function that gives top 10 country names in a given month

give_me_top10_names(df = cm_master, month_value = 510, var_name = 'ged_best_sb')

In [ ]:
#function 3, function that gives the cumulative top countries for a given period of time
#e.g., if the sum of the variable from monthA to monthB are taken, which countries are the top
#can modify which top X value you want, just the top 1 or top 5, etc.
#note that setting a range of months to 1 and the number of top wanted to 10 gives same result as above
#time_index is what variable is used as an index for time

#note same output as above
display(give_me_topX_country_id_cumulative(df=cm_master, time_index = 'month_id', number_wanted = 10,
                                           variable = 'ged_best_sb', start = 510, end = 510))

#top 5 for month 501-510
display(give_me_topX_country_id_cumulative(df=cm_master, time_index = 'month_id', number_wanted = 5,
                                           variable = 'ged_best_sb', start = 501, end = 510))


In [ ]:
#function 4, same function as function 3 but outputs names and not country_ids
#note same output as above
display(give_me_topX_country_names_cumulative(df=cm_master, time_index = 'month_id', number_wanted = 10,
                                           variable = 'ged_best_sb', start = 510, end = 510))

#top 5 for month 501-510
display(give_me_topX_country_names_cumulative(df=cm_master, time_index = 'month_id', number_wanted = 5,
                                           variable = 'ged_best_sb', start = 501, end = 510))


## Determines violence type of a variable

This function requres a very specific variable naming convention. 
This requires that the violence type is listed last and log transformation level is first

e.g., ln1_ged_best_sb = log transformed 1 variable, state-based violence

In [ ]:
#function 1, give you a human output for violence type, best used in titles, writing in graphic output
display(find_the_violence_type('ged_best_sb'))
display(find_the_violence_type('dummy_dummy_ns'))
display(find_the_violence_type('ln1_dummy_dummmy_os'))

In [ ]:
#function 2, give you a subscript name, best used in savefiles
display(give_me_violence_string_label_only('ged_best_sb'))
display(give_me_violence_string_label_only('dummy_dummy_ns'))
display(give_me_violence_string_label_only('ln1_dummy_dummmy_os'))

## Putting everything together

In [ ]:
## Example of everything used above together

import geopandas as gpd
import sqlalchemy as sa
from ingester3.config import source_db_path

from views_mapper2.mapper2 import *

from views_mapper2.dictionary_writer import *
from views_mapper2.BBoxWriter import *

engine = sa.create_engine(source_db_path)
gdf_ci_master = gpd.GeoDataFrame.from_postgis(
    "SELECT id as country_id, name, in_africa, in_me, geom FROM prod.country",
    engine,
    geom_col='geom'
)
gdf_ci_master = gdf_ci_master.to_crs(4326)

In [ ]:
LabelWriterFolder = working_folder + '/LabelWriterShowOff'
make_a_folder(LabelWriterFolder)

data= cm_master.copy()

#actuals data transformation
data['ln1_ged_best_sb'] = np.log1p(data['ged_best_sb'])
data['ln2_ged_best_sb'] = np.log1p(data['ln1_ged_best_sb'])

#prep data
gdf = gdf_ci_master.copy()
data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#this adds options for multiple month graphing, in this case we are only interested in EndOfHistory
steps_loop = [410, 500] 

#specify steps of interest and variables of interest
variable_loop = {'ged_best_sb', 'ln1_ged_best_sb', 'ln2_ged_best_sb'}


#specify regions to map
#requires either pre-set names like globe, ame or africa, or country_id in a numerical/integer format
#if you do not know country_id, can use a country_id identifying function
#name_mid2cid takes country name as string and month_id as integer
#name_date2cid takes country name as string and date as iso_string
geo_coverage_loop = ['globe', 'ame', name_mid2cid('Ukraine', 510), name_date2cid('Nigeria', '2022-08-01')]


#transparency setting, on a 0-100% value, 0 for invisible and 1 for 100% opacity
transparency_loop = [1]

#background loop, in this case None
background_loop = [None]

for step in steps_loop:
    for variable in variable_loop:
        for region_name in geo_coverage_loop:
                for transparency in transparency_loop:
                    for background in background_loop:
                        data_run = gdf.loc[step]
                        var_run = variable
                        var_name_run = find_the_violence_type(variable)
                        var_run_savefile = give_me_violence_string_label_only(variable)
                        bbox_run = bbox_from_cid(region_name)
                        
                        #the function below is explained separately in BBoxWriter and DictionaryWriter sections
                        
                        violence_dictionary = find_the_dictionary(variable, dictionary_0 = dictionary_stand, 
                                                                  dictionary_log1 = dictionary_stand_1p_10k, 
                                                                 dictionary_log2 = dictionary_stand_2p_10k)
                        for dictionary_name, dictionary in violence_dictionary.items():
                            dict_run_name = dictionary_name
                            dict_run = dictionary
                            
                        transparency_run = transparency
                        background_run = background
                        
                        
                        #this turns on experimental font for cases without a background names and at country level
                        #this is where it tends to work the best due to limited number of countries that are well spread
                        #modify size to get better results and avoid collision of labels
                        if str(region_name) in ('globe', 'ame', 'africa') or str(background) in ('OpenTopoMap', 'OpenStreetMap','StamenLite', 'StamenWatercolor','CartoDBPositron', 'CartoDBVoyager'): 
                            views_experimental_labels_font_size = None
                        else: views_experimental_labels_font_size = 15
                            
                            
                        #the following adjusts the size of the textbox, may need to be modified if the size of the output is changed
                        if str(region_name) in('globe'):
                            textbox_font_size = 10
                        elif str(region_name) in ('ame', 'africa'):
                            textbox_font_size = 20
                        else:textbox_font_size = 15
                        
                        title_run = f'Actual fatalities in {str(vid2date(step))}, {var_name_run}, background {background_run}'
                        textbox = f'Name: {var_run}_{str(step)}'
                        savefile = f'{LabelWriterFolder}/ActualsMap_continuous_cm{var_run_savefile}_{region_name}_rainbow_s0_month{step}_{dict_run_name}_trans{transparency_run}_{background_run}.png'
                        
                        map_run=Mapper2(
                            width=20,
                            height=20,
                            frame_on=True,
                            title=title_run,
                            bbox=bbox_run
                        ).add_layer(
                            gdf=data_run,
                            map_dictionary=dict_run,
                            cmap='rainbow',
                            edgecolor="black",
                            linewidth=0.5,
                            column=var_run,
                            transparency = transparency_run,
                            background = background_run,
                            views_experimental_labels = views_experimental_labels_font_size
                        ).add_views_textbox(
                            text=textbox,
                            textsize=textbox_font_size)
                        
                        map_run.save(savefile)